In [1]:
# https://github.com/evanm31/p4k-scraper/blob/master/data/scrapefork.py

In [2]:
# !python -m pip install retry
# !python -m pip install backoff

#### Import packages and set up enviornment

In [3]:
import urllib
import re
import time
import retry
import backoff
import pandas as pd
import requests
import datetime
import os
from bs4 import BeautifulSoup

import pitchfork_ratings_def # my custom functions from github

if os.name == "posix":
    data_path = "/Users/kylezengo/Desktop/DS/Music Ratings/"

#### Load historic data

In [4]:
# Load list of album review links
links_file_list = os.listdir(data_path+"links")

links_list = []
for i in links_file_list:
    links = open(data_path+"links/"+i)
    links = links.read().splitlines()
    
    links_list += links

links_list = list(set(links_list))
links_list = [x for x in links_list if x.startswith("https")]
print(f'{len(links_list)} album review links')

26806 album review links


In [5]:
# # Not sure what to do with this yet...
# pd.read_csv(data_path+"reviews/pitchfork_reviews_hist.csv").head()

In [6]:
reviews_file_list = os.listdir(data_path+"reviews")
reviews_file_list.remove('pitchfork_reviews_hist.csv')

reviews = []
for i in reviews_file_list:
    df = pd.read_csv(data_path+"reviews/"+i)
    reviews.append(df)

reviews = pd.concat(reviews).drop_duplicates().reset_index(drop=True)
print(f'{reviews.shape[0]} album reviews in data frame')
reviews.head()

26806 album reviews in data frame


,artist,album,score,genre,review,best,reviewed_date,link
0,Harmony,Gossip,5.3,Pop/R&B,Dr. Seuss is trashed at the club—can you pleaa...,0,"October 28, 2024",https://pitchfork.com/reviews/albums/harmony-g...
1,Mediopicky,el precio de la yuca / Bexaco y rico,7.9,"Electronic Reviewed: October 28, 2024 Are toda...",Are today’s urbano artists getting too big for...,0,"October 28, 2024",https://pitchfork.com/reviews/albums/mediopick...
2,Fievel Is Glauque,Rong Weicknes,8.0,Experimental / Jazz,Fievel Is Glauque conjure up visions of the bu...,0,"October 28, 2024",https://pitchfork.com/reviews/albums/fievel-is...
3,Missy Elliott,Da Real World,8.0,Rap,"“Have you ever had a dream, Neo, that you were...",0,"October 27, 2024",https://pitchfork.com/reviews/albums/missy-ell...
4,Boduf Songs,Lion Devours the Sun,7.0,Folk/Country,Mat Sweet lives in a world that's interesting ...,0,"October 16, 2006",https://pitchfork.com/reviews/albums/9491-lion...


In [7]:
# do list for links n

#### Get the link of each album review and save off to text file

In [8]:
%%time
# 3-4s per page (5-10min to run)

new_links = []
i = 1
while requests.get("https://pitchfork.com/reviews/albums/?page="+str(i)).status_code==200:
    req = urllib.request.Request("https://pitchfork.com/reviews/albums/?page=" + str(i))
    html_page = pitchfork_ratings_def.urlopen_with_retry(req)
    print(f'{i}: html_page status code {html_page.getcode()}')

    soup = BeautifulSoup(html_page, "lxml")
    
    for link in soup.findAll('a'):
        link_get_href = link.get('href')
        if link_get_href == '/reviews/albums/':
            pass
        elif link_get_href.startswith( '/reviews/albums/?genre=' ):
            pass
        elif link_get_href.startswith( '/reviews/albums/' ):
            new_links.append("https://pitchfork.com"+link_get_href)
    i+=1

new_links = list(set(new_links))

1: html_page status code 200
2: html_page status code 200
3: html_page status code 200
4: html_page status code 200
5: html_page status code 200
6: html_page status code 200
7: html_page status code 200
8: html_page status code 200
9: html_page status code 200
10: html_page status code 200
11: html_page status code 200
12: html_page status code 200
13: html_page status code 200
14: html_page status code 200
15: html_page status code 200
16: html_page status code 200
17: html_page status code 200
18: html_page status code 200
19: html_page status code 200
20: html_page status code 200
21: html_page status code 200
22: html_page status code 200
23: html_page status code 200
24: html_page status code 200
25: html_page status code 200
26: html_page status code 200
27: html_page status code 200
28: html_page status code 200
29: html_page status code 200
30: html_page status code 200
31: html_page status code 200
32: html_page status code 200
33: html_page status code 200
34: html_page statu

In [9]:
len(new_links)

9982

In [10]:
# Save list since takes time to generate
with open(f'{data_path}links/new_links_{datetime.datetime.now().date()}.txt', 'w') as f:
    for item in new_links:
        f.write("%s\n" % item)

#### Scrape reviews for each album and save to csv

In [11]:
links_list_with_new = links_list+new_links
    
checked_links = set(reviews['link'])
not_checked_links = [x for x in links_list_with_new if x not in checked_links]
print(f'{len(not_checked_links)} links without review data')

0 links without review data


In [12]:
# # test links
# new_links = ["https://pitchfork.com/reviews/albums/james-devane-searching/",
#              "https://pitchfork.com/reviews/albums/blood-incantation-absolute-elsewhere/",
#              "https://pitchfork.com/reviews/albums/body-meat-starchris/",
#              "https://pitchfork.com/reviews/albums/fred-again-actual-life-3-january-1-september-9-2022/",
#              "https://pitchfork.com/reviews/albums/100-gecs-10000-gecs/"]

In [13]:
%%time
# 2s? per page (long time to run >1 hr)

dat = []
for idx, i in enumerate(not_checked_links):
    print(f'{idx}: {i}')
    dat.append(pitchfork_ratings_def.gather_info(i))

new_reviews = pd.concat(dat).reset_index(drop=True)

ValueError: No objects to concatenate

In [14]:
# new_reviews = pd.concat(dat).reset_index(drop=True)
# new_reviews

In [15]:
# # Testing errors
# album_link = "https://pitchfork.com/reviews/albums/various-artists-even-the-forest-hums-ukrainian-sonic-archives/" 
# gather_info(album_link)

# page = requests.get(album_link) #request URL
# soup = BeautifulSoup(page.content, 'html.parser') #parse with beautifulsoup
# soup

In [16]:
new_reviews.to_csv(f'{data_path}reviews/new_reviews_{datetime.datetime.now().date()}.csv',index=False)

NameError: name 'new_reviews' is not defined